In [3]:
!export HF_ENDPOINT=https://hf-mirror.com
!export CUDA_VISIBLE_DEVICES=3

In [5]:
# package import
from torch import Tensor
from transformer_lens import utils
from functools import partial
from jaxtyping import Int, Float
import torch
# device setup
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cuda


In [6]:
from dotenv import load_dotenv
load_dotenv("/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/.env")

True

In [7]:
from transformer_lens import HookedTransformer
import numpy as np
import torch
import plotly_express as px

from transformer_lens import HookedTransformer

# Model Loading

from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

# Virtual Weight / Feature Statistics Functions
from sae_lens.analysis.feature_statistics import (
    get_all_stats_dfs,
    get_W_U_W_dec_stats_df,
)

# Enrichment Analysis Functions
from sae_lens.analysis.tsea import (
    get_enrichment_df,
    manhattan_plot_enrichment_scores,
    plot_top_k_feature_projections_by_token_and_category,
)
from sae_lens.analysis.tsea import (
    get_baby_name_sets,
    get_letter_gene_sets,
    generate_pos_sets,
    get_test_gene_sets,
    get_gene_set_from_regex,
)
import os
os.environ['HF_ENDPOINT']="https://hf-mirror.com"
# model = HookedTransformer.from_pretrained(
#     "tiny-stories-1L-21M"
# )  # This will wrap huggingface models and has lots of nice utilities.


model = HookedTransformer.from_pretrained("gpt2-small",device=device)

Loaded pretrained model gpt2-small into HookedTransformer


In [8]:
from sae_lens import SAE
layer=10

gpt2_small_sparse_autoencoders = {}
gpt2_small_sae_sparsities = {}


sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gpt2-small-res-jb",  # see other options in sae_lens/pretrained_saes.yaml
    sae_id=f"blocks.{layer}.hook_resid_pre",  # won't always be a hook point
    device=device
)
gpt2_small_sparse_autoencoders[f"blocks.{layer}.hook_resid_pre"] = sae
gpt2_small_sae_sparsities[f"blocks.{layer}.hook_resid_pre"] = sparsity

/home/ckqsudo/miniconda3/envs/SAE/lib/python3.11/site-packages/sae_lens/sae.py:145: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [10]:
from datasets import load_dataset
dataset = load_dataset("/home/ckqsudo/code2024/0dataset/emotional_classify/multiclass-sentiment-analysis")

In [11]:
dataset["train"]= dataset['train'].shuffle(seed=42)  # seed 用于固定随机性



In [12]:
train_data_text=dataset["train"]# 假设数据集是训练集（train），筛选 labels = 1 的数据
neg_train_set = dataset['train'].filter(lambda example: example['label'] == 0).select(range(1000))

pos_train_set=dataset['train'].filter(lambda example: example['label'] == 2).select(range(1000))

In [13]:
neu_train_set=dataset['train'].filter(lambda example: example['label'] == 1).select(range(1000))

In [14]:
len(neg_train_set),len(pos_train_set)

(1000, 1000)

In [15]:
neg_train_set["text"][1],pos_train_set["text"][1]

('needs someone to explain lambda calculus to him! :(', 'Amazing!!!!')

In [16]:
hook_point = sae.cfg.hook_name

In [19]:
sv_logits, cache = model.run_with_cache(neg_train_set["text"][:], prepend_bos=True,device=device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 566.00 MiB. GPU 0 has a total capacity of 23.65 GiB of which 423.62 MiB is free. Including non-PyTorch memory, this process has 23.22 GiB memory in use. Of the allocated memory 22.53 GiB is allocated by PyTorch, and 251.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [18]:
sv_logits.shape

torch.Size([50, 79, 50257])

In [92]:
batch_hidden_states=cache[hook_point]

In [93]:
batch_hidden_states.device

device(type='cuda', index=0)

In [105]:
batch_hidden_states.shape

torch.Size([50, 79, 768])

In [152]:
import torch

# 定义一个三维 Tensor
tensor = torch.tensor([
    [
    [0, 1, 0, 2],
    [3, 0, 0, 4],
    [0, 0, 0, 0]],

                       [[5, 0, 6, 0],
                        [0, 7, 0, 8],
                        [9, 0, 0, 0]]])

# 统计在 A 和 B 两个维度上不为 0 的元素，输出 C 维度的向量
nonzero_counts_C = (tensor != 0).sum(dim=(0, 1))
# # 1. 统计非零元素的数量
# latents_count = (batch_latents != 0).sum(dim=(0, 1))

# 打印结果
print(nonzero_counts_C,tensor.shape)

tensor([3, 2, 1, 3]) torch.Size([2, 3, 4])


In [153]:
from matplotlib import axis
import torch
batch_latents=[]
for data_idx in range(0,batch_hidden_states.shape[0]):
    hidden_state=batch_hidden_states[0]
    # hidden_state.to("cuda")
    latents = sae.encode(hidden_state)
    batch_latents.append(latents)
batch_latents=torch.stack(batch_latents,axis=0)


In [101]:
# import torch

# # 定义矩阵
# matrix = torch.zeros(4,2)
# matrix[2,1]=4
# matrix[3,1]=9
# matrix[1,0]=-1
# # 统计每一列中不为 0 的元素的个数
# nonzero_counts = (matrix != 0).sum(dim=0)

# # 打印结果
# print(nonzero_counts)

tensor([1, 2])


In [154]:
batch_latents.shape

torch.Size([50, 79, 24576])

In [155]:
batch_latents.shape

torch.Size([50, 79, 24576])

In [196]:

# 1. 统计非零元素的数量
act_cnt = (batch_latents != 0).sum(dim=(0, 1))

# 2. 计算非零元素的总和
nz_sum = torch.where(batch_latents != 0, batch_latents, torch.tensor(0.0)).sum(dim=(0, 1))

# 3. 计算非零元素的均值
nz_mean = torch.where(act_cnt != 0, nz_sum / act_cnt, torch.tensor(0.0))

In [200]:
nz_act_val, nz_val_indices= torch.topk(nz_mean, 100)

In [201]:
nz_act_val

tensor([553.5056, 494.7841, 399.6656, 383.3964, 241.6597, 191.9576, 169.6581,
        118.1690,  79.8443,  70.9105,  57.2500,  52.8709,  51.4807,  47.7746,
         44.4511,  35.5774,  32.7766,  32.3756,  32.1801,  31.3638,  31.0149,
         30.5907,  30.2827,  27.8037,  26.9767,  25.3160,  25.1925,  24.2746,
         23.5312,  22.7644,  22.3964,  22.0138,  21.5470,  21.2496,  21.0829,
         20.0239,  19.5796,  19.5531,  19.4559,  19.3109,  17.9848,  17.8117,
         17.1467,  16.8172,  16.6261,  16.6228,  15.0596,  14.9625,  14.2406,
         13.6629,  13.4630,  12.8589,  12.8523,  12.8316,  12.6341,  12.5975,
         12.3023,  12.2855,  12.0564,  11.6732,  11.1929,  11.1421,  10.5319,
         10.4749,  10.4572,  10.4009,  10.3488,  10.3365,  10.3348,  10.1578,
         10.0861,   9.7485,   9.5725,   9.4898,   9.4115,   9.3789,   8.9695,
          8.9366,   8.8940,   8.6021,   8.5531,   8.3255,   8.2921,   8.2480,
          8.2471,   8.2084,   8.0124,   7.7982,   7.7444,   7.70

In [202]:
nz_val_indices

tensor([ 1794,  4268, 23726,  9576, 16384, 12421, 19615, 14387, 11539,  2858,
        12235, 15479, 10658, 18169, 14130, 17373,  4467, 19022, 13299, 23934,
        20074, 14176,  9855, 19157, 24405, 19522, 11332, 11448,  4885,   398,
        12475, 15738, 15821,   986,  9478, 10113,  2155, 22522,  4565,    19,
        20884, 17822,  3906, 18274, 23258,  4212,  5128, 14034, 10866,  1701,
        20439, 13022, 14700, 16597,  8944, 14048, 19687, 13290, 22657, 13819,
        13309, 23061,  2501, 15835, 23508, 22918, 18593, 24398,  7000,  5927,
         6333, 23006,  1739, 21399, 18643, 24211, 11011, 21996, 23857,  8494,
        23903, 13672,  8353, 11566,  2932,  3137, 10275,  7641, 20471, 19601,
        19734,  2918,  4021,  9009, 23652, 20517, 19691, 16140, 17532,  2244],
       device='cuda:0')

In [203]:
nz_cnt, cnt_indices = torch.topk(act_cnt, 200)

# nonzero_indices = torch.nonzero(latents_count > 0).squeeze()

In [216]:
neg_acti_indices=nz_val_indices[torch.isin(nz_val_indices,cnt_indices)]

In [217]:
import torch

# 创建两个 Tensor
tensor1 = torch.tensor([1, 2, 3, 4, 5])
tensor2 = torch.tensor([4, 5, 6, 7, 8,9])

# 检查 tensor1 中的元素是否在 tensor2 中
mask = torch.isin(tensor1, tensor2)

# 提取重复元素
intersection = tensor1[mask]

# 打印结果
print("重复元素:", intersection,mask.shape)

重复元素: tensor([4, 5]) torch.Size([5])


In [218]:
def check_acts_by_index(prompt,acts_idx,hook_point):
    sv_logits, cache = model.run_with_cache(prompt, prepend_bos=True)
    # 转换为对应的令牌表示
    tokens = model.to_tokens(prompt)
    print(tokens)

    # get the feature activations from our SAE 获取特定位置的中间状态,并对中间状态进行编码，得到特征激活
    sv_feature_acts = sae.encode(cache[hook_point])
    print(sv_feature_acts.shape)
    return sv_feature_acts[:,:,acts_idx-1:acts_idx+1]


In [236]:
steering_vectors=Tensor.mean(sae.W_dec[neg_acti_indices],axis=0)
# d_hidden=sae.W_dec.shape[1]
# d_latent=sae.W_dec.shape[0]

In [237]:
steering_vectors.shape

torch.Size([768])

In [238]:
# 

from ast import Raise


def steering_hook(resid_pre, hook):
    if resid_pre.shape[1] == 1:
        return

    # position = steer_prompt_id.shape[1]
    # raise ValueError(f"steering{resid_pre.shape}")
    if steering_on:
        # using our steering vector and applying the coefficient
        # position=steer_prompt_id.shape[-1]
        resid_pre[:, :, :] += coeff * steering_vectors


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            stop_at_eos=False,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=50,
            do_sample=True,
            **kwargs,
        )
    return result
def run_generate(example_prompt):
    model.reset_hooks()
    editing_hooks = [(f"blocks.{layer}.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt] * 3, editing_hooks, seed=None, **sampling_kwargs
    )# batch=3, e

    # Print results, removing the ugly beginning of sequence token
    res_str = model.to_string(res[:, 1:])
    print(("\n\n" + "-" * 80 + "\n\n").join(res_str))

In [239]:
sampling_kwargs = dict(temperature=1.0, top_p=0.5, freq_penalty=1.0)

In [240]:
example_prompt=neu_train_set["text"][1]

In [241]:
example_prompt

'Is listening to the new jonas brothers song'

In [246]:
steering_on=False
example_prompt="I feel"
coeff=0
run_generate(example_prompt=example_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

I feel like I am doing a fair bit of damage here, but if you want to read about it in detail, I will give you a short synopsis.

In order to gain some level of clarity from the world around me, the gamer community has

--------------------------------------------------------------------------------

I feel like this might be the beginning of something big. The album is not going to last long. There are many things that could have been done better, but they didn't do them. It's too early to tell what will happen next, but I

--------------------------------------------------------------------------------

I feel like I'm missing out on something. I love music, so when I saw the "Little Helper" video, it was absolutely amazing. It was so surreal to see the scene of my favorite song ever played in a big venue and hear how


In [247]:
steering_on=True
# example_prompt="hey! I feel"
coeff=500
run_generate(example_prompt=example_prompt)

  0%|          | 0/50 [00:00<?, ?it/s]

I feel just like the most boring and unexplainable thing ever to happen to me.

The other day I got a call from my husband, who was reading about something on the internet that I had been thinking about for quite some time. It sounded

--------------------------------------------------------------------------------

I feel sorry for the English press, they have been trying to turn me into a racist. I feel sorry for them because I'm an immigrant.

They've used my identity as a white man and everything about me to try and get me in trouble

--------------------------------------------------------------------------------

I feel sorry for you. I'm sorry that you've become so involved in my life, and the things I did to help me get where I am today.

And I know that you are an inspiration to me. You're not only a mentor
